# 🎨 Stable Diffusion WebUI for Google Colab

**Complete image generation platform with API keys management, model downloads, and advanced features**

This notebook provides:
- ✅ HuggingFace & Civitai API key configuration
- ✅ Checkpoint and LoRA model downloads
- ✅ Full image generation with advanced parameters
- ✅ Google Drive integration for backup
- ✅ GPU acceleration and memory optimization

---

## 1️⃣ Initialize Dependencies and Environment

In [ ]:
# Install required packages
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors
!pip install -q huggingface-hub omegaconf einops requests
!pip install -q opencv-python-headless pillow numpy scipy
!pip install -q ipywidgets google-auth-oauthlib google-auth-httplib2 google-api-python-client
!pip install -q flask flask-socketio python-socketio

print("✅ All packages installed successfully!")

In [ ]:
import os
import sys
import json
import time
import torch
import numpy as np
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import PIL for image handling
from PIL import Image
import io
import base64

# Import diffusers
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionXLPipeline,
    EulerDiscreteScheduler,
    PNDMScheduler,
    DDIMScheduler,
    AutoencoderKL
)
from transformers import CLIPTokenizer, CLIPTextModel

# Import huggingface hub
from huggingface_hub import hf_hub_download, list_repo_files

# Import ipywidgets for GUI
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Setup
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Create directories
Path('./models/checkpoints').mkdir(parents=True, exist_ok=True)
Path('./models/loras').mkdir(parents=True, exist_ok=True)
Path('./models/vaes').mkdir(parents=True, exist_ok=True)
Path('./outputs').mkdir(parents=True, exist_ok=True)

print("\n✅ Directories created successfully!")

## 2️⃣ API Keys Configuration

In [ ]:
class APIKeysManager:
    """Manage API keys for HuggingFace and Civitai"""
    
    def __init__(self):
        self.hf_token = os.getenv('HF_TOKEN', '')
        self.civitai_key = os.getenv('CIVITAI_API_KEY', '')
        self.status = {'hf': False, 'civitai': False}
    
    def set_hf_token(self, token: str) -> bool:
        """Set HuggingFace token"""
        if token.strip():
            self.hf_token = token.strip()
            os.environ['HF_TOKEN'] = self.hf_token
            self.status['hf'] = True
            return True
        return False
    
    def set_civitai_key(self, key: str) -> bool:
        """Set Civitai API key"""
        if key.strip():
            self.civitai_key = key.strip()
            os.environ['CIVITAI_API_KEY'] = self.civitai_key
            self.status['civitai'] = True
            return True
        return False
    
    def get_headers(self, service: str = 'hf') -> Dict:
        """Get request headers for API calls"""
        if service == 'hf' and self.hf_token:
            return {'Authorization': f'Bearer {self.hf_token}'}
        elif service == 'civitai' and self.civitai_key:
            return {'Authorization': f'Bearer {self.civitai_key}'}
        return {}

# Initialize API Manager
api_manager = APIKeysManager()

# Create GUI for API keys
hf_token_input = widgets.Password(
    placeholder='Enter HuggingFace token',
    description='HF Token:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

civitai_key_input = widgets.Password(
    placeholder='Enter Civitai API key',
    description='Civitai Key:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

hf_status = widgets.HTML(value="<i style='color:gray'>Not configured</i>")
civitai_status = widgets.HTML(value="<i style='color:gray'>Not configured</i>")

def save_hf_token(btn):
    if api_manager.set_hf_token(hf_token_input.value):
        hf_status.value = "<b style='color:green'>✓ Configured</b>"
    else:
        hf_status.value = "<b style='color:red'>✗ Invalid token</b>"

def save_civitai_key(btn):
    if api_manager.set_civitai_key(civitai_key_input.value):
        civitai_status.value = "<b style='color:green'>✓ Configured</b>"
    else:
        civitai_status.value = "<b style='color:red'>✗ Invalid key</b>"

hf_btn = widgets.Button(description='Save HF Token', button_style='info')
hf_btn.on_click(save_hf_token)

civitai_btn = widgets.Button(description='Save Civitai Key', button_style='info')
civitai_btn.on_click(save_civitai_key)

print("🔑 API Keys Configuration\n")
display(
    widgets.VBox([
        widgets.HTML("<h3>HuggingFace Token</h3>"),
        widgets.HBox([hf_token_input, hf_btn]),
        hf_status,
        widgets.HTML("<small><a href='https://huggingface.co/settings/tokens' target='_blank'>Get token from HuggingFace</a></small>"),
        
        widgets.HTML("<hr>"),
        widgets.HTML("<h3>Civitai API Key</h3>"),
        widgets.HBox([civitai_key_input, civitai_btn]),
        civitai_status,
        widgets.HTML("<small><a href='https://civitai.com/user/account' target='_blank'>Get API key from Civitai</a></small>"),
    ])
)

## 3️⃣ Model Download Management

In [ ]:
class ModelDownloader:
    """Handle model downloads from various sources"""
    
    def __init__(self):
        self.checkpoint_dir = Path('./models/checkpoints')
        self.lora_dir = Path('./models/loras')
        self.vae_dir = Path('./models/vaes')
    
    def download_from_huggingface(self, model_id: str, hf_token: str = None) -> Optional[Path]:
        """Download model from HuggingFace Hub"""
        try:
            filepath = hf_hub_download(
                repo_id=model_id,
                filename="model.safetensors",
                cache_dir=str(self.checkpoint_dir),
                token=hf_token,
                force_download=False
            )
            print(f"✅ Downloaded: {Path(filepath).name}")
            return Path(filepath)
        except Exception as e:
            print(f"❌ Error: {e}")
            return None
    
    def download_from_url(self, url: str, output_path: Path) -> bool:
        """Download model from direct URL"""
        import requests
        try:
            response = requests.get(url, stream=True, timeout=30)
            response.raise_for_status()
            
            total_size = int(response.headers.get('content-length', 0))
            with open(output_path, 'wb') as f:
                downloaded = 0
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        if total_size:
                            progress = (downloaded / total_size) * 100
                            print(f"\rDownloading... {progress:.1f}%", end='', flush=True)
            print(f"\n✅ Downloaded: {output_path.name}")
            return True
        except Exception as e:
            print(f"\n❌ Error: {e}")
            return False
    
    def list_models(self) -> Dict[str, List[str]]:
        """List all available models"""
        models = {
            'checkpoints': [f.name for f in self.checkpoint_dir.glob('*.safetensors') if f.is_file()],
            'loras': [f.name for f in self.lora_dir.glob('*.safetensors') if f.is_file()],
            'vaes': [f.name for f in self.vae_dir.glob('*.safetensors') if f.is_file()]
        }
        return models
    
    def get_model_size(self, filepath: Path) -> str:
        """Get human-readable file size"""
        size_bytes = filepath.stat().st_size
        for unit in ['B', 'KB', 'MB', 'GB']:
            if size_bytes < 1024:
                return f"{size_bytes:.2f} {unit}"
            size_bytes /= 1024
        return f"{size_bytes:.2f} TB"

# Initialize downloader
downloader = ModelDownloader()

# Create GUI
checkpoint_url = widgets.Text(
    placeholder='e.g., runwayml/stable-diffusion-v1-5 or https://...',
    description='Checkpoint:',
    layout=widgets.Layout(width='500px')
)

lora_url = widgets.Text(
    placeholder='HuggingFace model ID or direct URL',
    description='LoRA:',
    layout=widgets.Layout(width='500px')
)

vae_url = widgets.Text(
    placeholder='VAE model URL',
    description='VAE:',
    layout=widgets.Layout(width='500px')
)

output_download = widgets.Output()

def download_checkpoint(btn):
    with output_download:
        clear_output()
        url = checkpoint_url.value.strip()
        if not url:
            print("❌ Please enter a checkpoint URL")
            return
        
        print(f"📥 Downloading checkpoint: {url}...")
        if '/' in url and not url.startswith('http'):
            downloader.download_from_huggingface(url, api_manager.hf_token)
        else:
            filename = url.split('/')[-1]
            downloader.download_from_url(url, Path(downloader.checkpoint_dir) / filename)

def download_lora(btn):
    with output_download:
        clear_output()
        url = lora_url.value.strip()
        if not url:
            print("❌ Please enter a LoRA URL")
            return
        
        print(f"📥 Downloading LoRA: {url}...")
        if '/' in url and not url.startswith('http'):
            downloader.download_from_huggingface(url, api_manager.hf_token)
        else:
            filename = url.split('/')[-1]
            downloader.download_from_url(url, Path(downloader.lora_dir) / filename)

def download_vae(btn):
    with output_download:
        clear_output()
        url = vae_url.value.strip()
        if not url:
            print("❌ Please enter a VAE URL")
            return
        
        print(f"📥 Downloading VAE: {url}...")
        if '/' in url and not url.startswith('http'):
            downloader.download_from_huggingface(url, api_manager.hf_token)
        else:
            filename = url.split('/')[-1]
            downloader.download_from_url(url, Path(downloader.vae_dir) / filename)

checkpoint_btn = widgets.Button(description='Download', button_style='success')
checkpoint_btn.on_click(download_checkpoint)

lora_btn = widgets.Button(description='Download', button_style='success')
lora_btn.on_click(download_lora)

vae_btn = widgets.Button(description='Download', button_style='success')
vae_btn.on_click(download_vae)

print("📦 Model Download Manager\n")
display(
    widgets.VBox([
        widgets.HTML("<h3>Download Checkpoint</h3>"),
        widgets.HBox([checkpoint_url, checkpoint_btn]),
        
        widgets.HTML("<h3>Download LoRA</h3>"),
        widgets.HBox([lora_url, lora_btn]),
        
        widgets.HTML("<h3>Download VAE</h3>"),
        widgets.HBox([vae_url, vae_btn]),
        
        output_download
    ])
)

# List available models
print("\n📂 Available Models:")
models = downloader.list_models()
for model_type, files in models.items():
    print(f"\n{model_type.upper()}:")
    if files:
        for f in files:
            path = {
                'checkpoints': downloader.checkpoint_dir,
                'loras': downloader.lora_dir,
                'vaes': downloader.vae_dir
            }[model_type] / f
            size = downloader.get_model_size(path)
            print(f"  - {f} ({size})")
    else:
        print(f"  (No models downloaded)")

## 4️⃣ Checkpoint and Model Loading

In [ ]:
class StableDiffusionManager:
    """Manage Stable Diffusion models and pipelines"""
    
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.pipe = None
        self.current_model = None
        self.loras = {}
        self.vae = None
    
    def load_model(self, model_id: str, hf_token: str = None, precision: str = 'fp16'):
        """Load a Stable Diffusion model"""
        try:
            print(f"🔄 Loading model: {model_id}...")
            
            torch_dtype = torch.float16 if precision == 'fp16' else torch.float32
            
            # Load based on model type
            if 'xl' in model_id.lower():
                pipe = StableDiffusionXLPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch_dtype,
                    use_safetensors=True,
                    use_auth_token=hf_token
                )
            else:
                pipe = StableDiffusionPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch_dtype,
                    use_safetensors=True,
                    use_auth_token=hf_token
                )
            
            pipe = pipe.to(self.device)
            
            # Enable optimizations
            pipe.enable_attention_slicing()
            if hasattr(pipe, 'enable_xformers_memory_efficient_attention'):
                try:
                    pipe.enable_xformers_memory_efficient_attention()
                except:
                    pass
            
            self.pipe = pipe
            self.current_model = model_id
            print(f"✅ Model loaded successfully!")
            return True
        
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return False
    
    def load_lora(self, lora_path: str, weight: float = 1.0):
        """Load LoRA model"""
        try:
            if not self.pipe:
                print("❌ Load a checkpoint first")
                return False
            
            self.pipe.load_lora_weights(lora_path)
            self.pipe.set_lora_device(self.device)
            self.pipe.fuse_lora(lora_scale=weight)
            print(f"✅ LoRA loaded: {Path(lora_path).name}")
            return True
        except Exception as e:
            print(f"❌ Error loading LoRA: {e}")
            return False
    
    def load_vae(self, vae_path: str):
        """Load custom VAE"""
        try:
            if not self.pipe:
                print("❌ Load a checkpoint first")
                return False
            
            vae = AutoencoderKL.from_single_file(vae_path)
            self.pipe.vae = vae.to(self.device)
            print(f"✅ VAE loaded: {Path(vae_path).name}")
            return True
        except Exception as e:
            print(f"❌ Error loading VAE: {e}")
            return False
    
    def generate(
        self,
        prompt: str,
        negative_prompt: str = "",
        height: int = 512,
        width: int = 512,
        num_inference_steps: int = 20,
        guidance_scale: float = 7.5,
        seed: int = -1,
        num_images: int = 1
    ) -> List[Image.Image]:
        """Generate images"""
        try:
            if not self.pipe:
                print("❌ Load a model first")
                return []
            
            if seed >= 0:
                generator = torch.Generator(device=self.device).manual_seed(seed)
            else:
                generator = None
            
            print(f"🎨 Generating {num_images} image(s)...")
            
            result = self.pipe(
                prompt,
                negative_prompt=negative_prompt,
                height=height,
                width=width,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                num_images_per_prompt=num_images,
                generator=generator
            )
            
            print(f"✅ Generation complete!")
            return result.images
        
        except Exception as e:
            print(f"❌ Generation error: {e}")
            return []

# Initialize manager
sd_manager = StableDiffusionManager()

# Create model selection GUI
model_options = [
    'runwayml/stable-diffusion-v1-5',
    'stabilityai/stable-diffusion-2-1',
    'stabilityai/stable-diffusion-xl-base-1.0'
]

model_dropdown = widgets.Dropdown(
    options=model_options,
    value=model_options[0],
    description='Model:',
    layout=widgets.Layout(width='500px')
)

precision_radio = widgets.RadioButtons(
    options=['fp16 (fast)', 'fp32 (accurate)'],
    description='Precision:'
)

load_output = widgets.Output()

def load_model_click(btn):
    with load_output:
        clear_output()
        precision = 'fp16' if 'fp16' in precision_radio.value else 'fp32'
        sd_manager.load_model(model_dropdown.value, api_manager.hf_token, precision)

load_btn = widgets.Button(description='Load Model', button_style='warning')
load_btn.on_click(load_model_click)

print("⚙️ Model Configuration\n")
display(
    widgets.VBox([
        widgets.HTML("<h3>Select Model</h3>"),
        model_dropdown,
        precision_radio,
        widgets.HBox([load_btn]),
        load_output
    ])
)

## 5️⃣ Generate Images with Full Parameter Control

In [ ]:
# Create generation GUI
prompt_input = widgets.Textarea(
    placeholder='Describe what you want to generate...',
    description='Prompt:',
    rows=4,
    layout=widgets.Layout(width='100%')
)

negative_prompt_input = widgets.Textarea(
    placeholder='What to avoid...',
    description='Negative:',
    rows=3,
    layout=widgets.Layout(width='100%')
)

# Generation parameters
width_input = widgets.IntSlider(min=64, max=2048, step=64, value=512, description='Width:')
height_input = widgets.IntSlider(min=64, max=2048, step=64, value=512, description='Height:')
steps_input = widgets.IntSlider(min=1, max=100, value=20, description='Steps:')
cfg_input = widgets.FloatSlider(min=1, max=20, step=0.5, value=7.5, description='CFG Scale:')
seed_input = widgets.IntSlider(min=-1, max=2147483647, value=-1, description='Seed:')
num_images_input = widgets.IntSlider(min=1, max=4, value=1, description='Num Images:')

sampler_dropdown = widgets.Dropdown(
    options=['euler', 'ddim', 'pndm', 'lms'],
    value='euler',
    description='Sampler:'
)

# Generation output
gen_output = widgets.Output()

def generate_click(btn):
    with gen_output:
        clear_output()
        
        prompt = prompt_input.value.strip()
        if not prompt:
            print("❌ Please enter a prompt")
            return
        
        if not sd_manager.pipe:
            print("❌ Load a model first")
            return
        
        # Set scheduler
        if sampler_dropdown.value == 'euler':
            sd_manager.pipe.scheduler = EulerDiscreteScheduler.from_config(sd_manager.pipe.scheduler.config)
        elif sampler_dropdown.value == 'ddim':
            sd_manager.pipe.scheduler = DDIMScheduler.from_config(sd_manager.pipe.scheduler.config)
        elif sampler_dropdown.value == 'pndm':
            sd_manager.pipe.scheduler = PNDMScheduler.from_config(sd_manager.pipe.scheduler.config)
        
        # Generate
        start_time = time.time()
        images = sd_manager.generate(
            prompt=prompt,
            negative_prompt=negative_prompt_input.value,
            height=int(height_input.value),
            width=int(width_input.value),
            num_inference_steps=int(steps_input.value),
            guidance_scale=float(cfg_input.value),
            seed=int(seed_input.value),
            num_images=int(num_images_input.value)
        )
        elapsed = time.time() - start_time
        
        if images:
            # Display images
            print(f"\n⏱️ Time: {elapsed:.2f}s\n")
            
            # Create grid
            cols = min(len(images), 2)
            rows = (len(images) + cols - 1) // cols
            grid = Image.new('RGB', (width_input.value * cols, height_input.value * rows))
            
            for idx, img in enumerate(images):
                x = (idx % cols) * width_input.value
                y = (idx // cols) * height_input.value
                grid.paste(img, (x, y))
                
                # Save
                output_path = Path('./outputs') / f"gen_{int(time.time())}_{idx}.png"
                img.save(output_path)
                print(f"💾 Saved: {output_path.name}")
            
            display(grid)

generate_btn = widgets.Button(description='🎨 Generate', button_style='success', button_width='150px')
generate_btn.on_click(generate_click)

print("🎨 Image Generation\n")
display(
    widgets.VBox([
        prompt_input,
        negative_prompt_input,
        widgets.HBox([width_input, height_input]),
        widgets.HBox([steps_input, cfg_input]),
        widgets.HBox([seed_input, num_images_input]),
        widgets.HBox([sampler_dropdown]),
        generate_btn,
        gen_output
    ])
)

## 🎯 Quick Examples

In [ ]:
# Example: Generate beautiful landscape
if sd_manager.pipe:
    example_prompt = "a beautiful landscape with mountains and lake, sunset, highly detailed, 4k resolution"
    example_negative = "low quality, blurry, distorted"
    
    print(f"📝 Example prompt: {example_prompt}")
    print(f"\nTo generate:")
    print(f"1. Paste the prompt above")
    print(f"2. Adjust parameters as needed")
    print(f"3. Click 'Generate' button")
else:
    print("⚠️ Load a model first to generate images")

## 📊 Gallery & Export

In [ ]:
def show_gallery():
    """Display all generated images"""
    output_dir = Path('./outputs')
    images = sorted(output_dir.glob('*.png'), key=lambda x: x.stat().st_mtime, reverse=True)
    
    if not images:
        print("📭 No images generated yet")
        return
    
    print(f"📸 Gallery ({len(images)} images)\n")
    
    # Display in grid
    cols = 3
    rows = (len(images) + cols - 1) // cols
    
    for i in range(0, len(images), cols):
        batch = images[i:i+cols]
        fig_width = 15
        fig_height = 5 * rows
        
        for img_path in batch:
            img = Image.open(img_path)
            print(f"\n{img_path.name} - {img.size}")
            display(img)

# Show gallery
show_gallery()

# Download all
def download_all():
    """Create downloadable archive"""
    import zipfile
    output_dir = Path('./outputs')
    if output_dir.exists() and list(output_dir.glob('*.png')):
        zip_path = Path('./generated_images.zip')
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for img in output_dir.glob('*.png'):
                zipf.write(img, img.name)
        print(f"✅ Archive created: {zip_path.name}")
        print(f"📦 Size: {zip_path.stat().st_size / (1024*1024):.2f} MB")
        return zip_path
    return None

print("\n💾 To download all generated images, run the next cell")

In [ ]:
# Download all images
zip_file = download_all()

if zip_file:
    from IPython.display import FileLink
    display(FileLink(str(zip_file)))

## 🚀 Tips & Tricks

- **Better Quality**: Increase steps (20-50) and CFG scale (7-15)
- **Faster Generation**: Lower steps (10-15) and use fp16 precision
- **LoRA Enhancement**: Download LoRA models for style customization
- **Memory Issues**: Enable memory efficient attention and CPU offload
- **Custom Seeds**: Use negative seed values for random generation
- **Batch Generation**: Set num_images > 1 to generate multiple variations

---

**Enjoy generating amazing images! 🎨**